# 학습데이터 만들기

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import matplotlib.pyplot as plt
import keyboard
import random

In [2]:
from moviepy.editor import *
video_clip = VideoFileClip('./Aespa_Girls_target.mp4').subclip(0, 79)
video_clip2 = VideoFileClip('./Aespa_Girls_user.mp4').subclip(0.5, 79)

In [3]:
# 포즈 감지 모델 초기화
mp_pose = mp.solutions.pose
pose_video = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.7,
                          min_tracking_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils

In [6]:
# 포즈 검출 함수
def detectPose(image_pose, pose, draw=False, display=False):
    
    original_image = image_pose.copy()
    
    image_in_RGB = cv2.cvtColor(image_pose, cv2.COLOR_BGR2RGB)
    
    resultant = pose.process(image_in_RGB)

    if resultant.pose_landmarks and draw:    

        mp_drawing.draw_landmarks(image=original_image, landmark_list=resultant.pose_landmarks,
                                  connections=mp_pose.POSE_CONNECTIONS,
                                  landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255,255,255),
                                                                               thickness=3, circle_radius=3),
                                  connection_drawing_spec=mp_drawing.DrawingSpec(color=(49,125,237),
                                                                               thickness=2, circle_radius=2))

    if display:
            
            plt.figure(figsize=[22,22])
            plt.subplot(121);plt.imshow(image_pose[:,:,::-1]);plt.title("Input Image");plt.axis('off');
            plt.subplot(122);plt.imshow(original_image[:,:,::-1]);plt.title("Pose detected Image");plt.axis('off');

    else:
        
        return original_image, resultant

In [7]:
import pandas as pd
df = pd.DataFrame()

In [8]:
for num, i in enumerate(np.arange(1, 79, 0.1)):
   
    img = video_clip.get_frame(i)
    img2 = video_clip2.get_frame(i)
    
    _, target_result = detectPose(img, pose_video)
    _, user_result = detectPose(img2, pose_video)
    
    img_target = cv2.resize(img, (0, 0), fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)
    img_user = cv2.resize(img2, (0, 0), fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)

    numpy_vertical = np.vstack((img_target, img_user))
    
    ver_cv = cv2.cvtColor(numpy_vertical, cv2.COLOR_BGR2RGB)
    cv2.imshow("target", ver_cv)
    cv2.waitKey()
    
    try:
        if keyboard.is_pressed('0'):            # 두 포즈가 다른 경우
            df.loc[num, 'target_lm'] = target_result.pose_world_landmarks.landmark
            df.loc[num, 'user_lm'] = target_result.pose_world_landmarks.landmark
            df.loc[num, 'label'] = 0
            
        if keyboard.is_pressed('1'):
            df.loc[num, 'target_lm'] = target_result.pose_world_landmarks.landmark
            df.loc[num, 'user_lm'] = target_result.pose_world_landmarks.landmark
            df.loc[num, 'label'] = 1
            
        if keyboard.is_pressed('esc'):
            break
    except:
        continue

cv2.destroyAllWindows()

In [ ]:
pd.concat([df[df['label'] == 1], df.iloc[id, :]]).to_csv("train.csv", index=False)

In [13]:
df.to_csv('aespa_girls_train.csv', index=False)

In [14]:
id

<function id(obj, /)>